In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Loaded environment variables from .env file.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "cais/Zephyr_RMU"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_type = Datasets.WMDP
dataset_config = DATASETS_DICT[dataset_type]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

retain_train = local_load_dataset(data_dir, dataset_config["retain_files"])
retain_val = local_load_dataset(data_dir, dataset_config["val_retain_files"])

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]

n_val_files = 4
max_length = 512

forget_train_1 = local_load_dataset(data_dir, unlearn_files[:n_val_files])
forget_train_2 = local_load_dataset(data_dir, unlearn_files[n_val_files:])

forget_val_1 = local_load_dataset(data_dir, val_unlearn_files[:n_val_files])
forget_val_2 = local_load_dataset(data_dir, val_unlearn_files[n_val_files:])

forget_train_1_records = process_corpus(forget_train_1, tokenizer, max_length)
forget_train_2_records = process_corpus(forget_train_2, tokenizer, max_length)
retain_train_records = process_corpus(retain_train, tokenizer, max_length)
forget_train_records = forget_train_1_records + forget_train_2_records

forget_train_mcq_1_records = process_mcq(
    forget_val_1, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_2_records = process_mcq(
    forget_val_2, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_records = forget_train_mcq_1_records + forget_train_mcq_2_records

forget_val_1_records = process_mcq(forget_val_1, tokenizer, max_length)
forget_val_2_records = process_mcq(forget_val_2, tokenizer, max_length)
retain_val_records = process_mcq(retain_val, tokenizer, max_length)
forget_val_records = forget_val_1_records + forget_val_2_records

Filter:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
if dataset_type == Datasets.RANDOM_BD:
    from relearn.unlearn.gd import train_gd

    model = train_gd(
        model,
        6,
        forget_train_records,
        retain_train_records,
        {
            "forget 1": forget_val_1_records,
            "forget 2": forget_val_2_records,
            "retain": retain_val_records,
        },
        forget_alpha=1,
        retain_both=True,
        batch_size=4,
        lr=1e-6,
        eval_at_start=True,
    )

In [5]:
import random


def prepare_model(model: AutoModelForCausalLM, layers: List[int]):
    params = []
    for layer in layers:
        module = model.model.layers[layer]
        module.requires_grad = False
        params.extend(module.parameters())
    return params


def reset_model(model: AutoModelForCausalLM):
    for layer in model.model.layers:
        layer.requires_grad = True
    return model


n_layers = len(model.model.layers)
layers = list(range(n_layers))
random.shuffle(layers)
layers_1 = layers[: n_layers // 2]
layers_2 = layers[n_layers // 2 :]

In [6]:
from relearn.unlearn.gd import train_gd

# unlearn first A
params = prepare_model(model, layers_1)

model = train_gd(
    model,
    1,
    forget_train_1_records,
    retain_train_records,
    {
        "forget 1": forget_val_1_records,
        "forget 2": forget_val_2_records,
        "retain": retain_val_records,
    },
    forget_alpha=1,
    retain_both=False,
    batch_size=4,
    lr=1e-5,
    eval_at_start=False,
    params=params,
)

model = reset_model(model)

50it [00:39,  1.25it/s, Loss=2.25, Forget Loss=0.0143, Retain Loss=2.24]  

Epoch 0, Step 49, Loss 2.2516019344329834, Forget Loss 0.014334678649902344, Retain Loss 2.237267255783081


100it [01:18,  1.26it/s, Loss=2.1, Forget Loss=0.0575, Retain Loss=2.04]  

Epoch 0, Step 99, Loss 2.0951669216156006, Forget Loss 0.057483918964862823, Retain Loss 2.0376830101013184


150it [01:57,  1.26it/s, Loss=2.02, Forget Loss=0.00953, Retain Loss=2.01] 

...

Epoch 0 forget 1 Accuracy: 0.2340764331210191


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 14.89it/s]


Epoch 0 forget 2 Accuracy: 0.2611464968152866


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:41<00:00,  9.41it/s]


Epoch 0 retain Accuracy: 0.556687898089172


In [7]:
from relearn.unlearn.gd import train_gd

# unlearn first B
params = prepare_model(model, layers_2)

model = train_gd(
    model,
    1,
    forget_train_2_records,
    retain_train_records,
    {
        "forget 1": forget_val_1_records,
        "forget 2": forget_val_2_records,
        "retain": retain_val_records,
    },
    forget_alpha=1,
    retain_both=False,
    batch_size=4,
    lr=1e-5,
    eval_at_start=False,
    params=params,
)

model = reset_model(model)

50it [00:38,  1.27it/s, Loss=1.42, Forget Loss=-0, Retain Loss=1.42]        

Epoch 0, Step 49, Loss 1.4236947298049927, Forget Loss -0.0, Retain Loss 1.4236947298049927


100it [01:17,  1.26it/s, Loss=0.851, Forget Loss=0.000852, Retain Loss=0.851]

Epoch 0, Step 99, Loss 0.8514068126678467, Forget Loss 0.0008519316324964166, Retain Loss 0.850554883480072


118it [01:31,  1.29it/s, Loss=1.23, Forget Loss=-0, Retain Loss=1.23]        
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 314/314 [00:20<00:00, 15.47it/s]


Epoch 0 forget 1 Accuracy: 0.21337579617834396


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 14.89it/s]


Epoch 0 forget 2 Accuracy: 0.17197452229299362


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:41<00:00,  9.40it/s]

Epoch 0 retain Accuracy: 0.5579617834394904


In [8]:
from relearn.attacks.rtt import train_ft

# relearm only A
model = train_ft(
    model,
    10,
    forget_train_mcq_1_records,
    {
        "forget 1": forget_val_1_records,
        "forget 2": forget_val_2_records,
        "retain": retain_val_records,
    },
    batch_size=4,
    lr=1e-6,
    eval_at_start=False,
)

50it [00:14,  3.58it/s]

Epoch 0, Step 49, Loss 0.12016799300909042


100it [00:29,  3.62it/s]

Epoch 0, Step 99, Loss 0.1275818794965744


150it [00:43,  3.41it/s]

...

Epoch 9 forget 1 Accuracy: 1.0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 14.90it/s]


Epoch 9 forget 2 Accuracy: 0.5095541401273885


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:41<00:00,  9.39it/s]

Epoch 9 retain Accuracy: 0.5515923566878981
